In [6]:
import pandas as pd 
import os

import pyspark.sql.functions as F
from pyspark.sql.types import *
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import Row
import pyarrow.parquet as pq
import pyarrow as pa

from bs4 import BeautifulSoup





os.environ['SPARK_HOME'] = "/home/veselovs/spark-3.2.1-bin-hadoop2.7"
os.environ['JAVA_HOME'] = "/home/veselovs/jdk-13.0.2"
spark = SparkSession.builder.getOrCreate()



23/01/12 13:25:29 WARN Utils: Your hostname, iccluster039 resolves to a loopback address: 127.0.1.1; using 10.90.38.15 instead (on interface ens786f0)
23/01/12 13:25:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/12 13:25:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/12 13:25:32 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/01/12 13:25:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/01/12 13:25:35 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [17]:
import sys
sys.path.append("code/helpers/")

In [23]:
from settings import WIKI_PAGES_DIR, DATA_DIR, EMBEDDING_DIR

import glob
files = glob.glob(WIKI_PAGES_DIR + "/*")

files = [k for k in files if k != "/dlabdata1/piccar£di/WikipediaPagesExtractor/WikiPages/CC-MAIN-20210307044328-20210307074328-00556.warc.gz.parquet"]


In [24]:
import sys

import importlib
sys.path.append("/scratch/venia/wiki_embedding_project")
from config.get import cfg
langlist = cfg["langlist"]

qid_pid = cfg["processed_data"]["matchings"]["Qid_pid"]["from_page_props"]


In [25]:
import glob 
files = glob.glob(DATA_DIR + "/*")

In [26]:
files = files[0:12]

In [27]:
raw = spark.read.load(files)

In [28]:
content = raw.select("content").limit(20).toPandas()


AnalysisException: cannot resolve 'content' given input columns: [domain, label, label_leaf, lang, uid];
'Project ['content]
+- Relation [domain#0,uid#1L,label#2,lang#3,label_leaf#4] parquet


In [ ]:
a = content["content"].values[2]

In [ ]:
# play with it to find weighting scheme

soup = BeautifulSoup(a)

In [ ]:
for link in soup.find_all('a'):
    l = link.get("href")
    if  l != None: 
        if "wiki" in l:
            d = l
            print(normalise_title(l))
            print(l)
        

In [ ]:
i = 0
while link.parent:
    print(i)
    link = link.parent
    i += 1

In [22]:
import urllib

def normalise_title(title):
    """ Replace _ with space, remove anchor, capitalize """
    title = title.split("/")[-1]
    title = urllib.parse.unquote(title)
    title = title.strip()
    if len(title) > 0:
        title = title[0].upper() + title[1:]
    n_title = title.replace("_", " ")
    # if '#' in n_title:
    #     n_title = n_title.split('#')[0]
    return n_title


In [51]:
for link in soup.find_all('a'):
    l = link.get("href")
    if  l != None: 
        if "wiki" in l:
            print(normalise_title(l))
            print(l)

Heteronormativität
https://de.wikipedia.org/wiki/Heteronormativit%C3%A4t


In [35]:
"http" in soup.find_all('a')[0].get("href")

True

In [31]:
df2 = pd.read_parquet("data/all_wikilinks_clean.parquet")

In [42]:
df2["domain"].nunique()

1475057

In [43]:
df2["url"].nunique()

24172414

In [46]:
df2["wiki_no_subset"] + df2[.nunique()

15163852

In [47]:
df2

,domain,url,wiki_link,subset,wiki_no_subset,wikidb,title
0,agorapoliton.gr,http://agorapoliton.gr/index.php?Itemid=62&cat...,https://en.wikipedia.org/wiki/Economy_of_North...,,https://en.wikipedia.org/wiki/Economy_of_North...,en,Economy of North Korea
1,amipsyche.com,http://amipsyche.com/6-romantic-ideas-for-newl...,https://en.wikipedia.org/wiki/Newlywed,,https://en.wikipedia.org/wiki/Newlywed,en,Newlywed
2,analysebrassens.com,http://analysebrassens.com?analysebrassens=d74...,http://fr.wikipedia.org/wiki/Homosexualit%C3%A...,#Pers.C3.A9cutions_des_r.C3.A9gimes_totalitaires,http://fr.wikipedia.org/wiki/Homosexualit%C3%A9,fr,Homosexualité
3,animejima.xyz,http://animejima.xyz/2008%E5%B9%B4%EF%BC%88%E5...,https://ja.wikipedia.org/wiki/%E3%81%A8%E3%81%...,,https://ja.wikipedia.org/wiki/%E3%81%A8%E3%81%...,ja,とある魔術の禁書目録 (アニメ)
4,animejima.xyz,http://animejima.xyz/2008%E5%B9%B4%EF%BC%88%E5...,https://ja.wikipedia.org/wiki/%E3%81%A8%E3%81%...,,https://ja.wikipedia.org/wiki/%E3%81%A8%E3%81%...,ja,とある魔術の禁書目録 (アニメ)
...,...,...,...,...,...,...,...
90805362,wirelessinfo.be,http://www.wirelessinfo.be/mikrotik-draadloos-...,https://en.wikipedia.org/wiki/Customer-premise...,,https://en.wikipedia.org/wiki/Customer-premise...,en,Customer-premises equipment
90805363,wirelessinfo.be,http://www.wirelessinfo.be/mikrotik-draadloos-...,https://nl.wikipedia.org/wiki/Service_Set_Iden...,,https://nl.wikipedia.org/wiki/Service_Set_Iden...,nl,Service Set Identifier
90805364,wirelessinfo.be,http://www.wirelessinfo.be/mikrotik-draadloos-...,https://en.wikipedia.org/wiki/Wireless_Interne...,,https://en.wikipedia.org/wiki/Wireless_Interne...,en,Wireless Internet service provider
90805365,wirelessinfo.be,http://www.wirelessinfo.be/mikrotik-draadloos-...,https://en.wikipedia.org/wiki/Wireless_Interne...,,https://en.wikipedia.org/wiki/Wireless_Interne...,en,Wireless Internet service provider


In [40]:
df2.groupby("wikidb")["title"].nunique().sort_values(ascending=False).head(40)

wikidb
en           3619416
de            824981
fr            692220
ja            656086
es            486811
ru            465283
wikipedia     385730
it            350046
pt            226713
sv            221530
nl            206727
zh            161789
pl            161395
cs            141455
uk            127204
id            119333
fa            112221
ca            110216
ar            107824
bg            103601
vi            101427
tr             99039
he             88318
hu             86229
ko             72900
no             72296
fi             71098
ro             65901
el             62544
da             58859
hak3           53226
commons        47878
sr             44833
eo             40107
eu             39772
sk             37492
hr             36819
th             36224
simple         36217
ms             34318
Name: title, dtype: int64

In [37]:
df2.groupby("wikidb")["url"].nunique().sort_values(ascending=False).head(40)

wikidb
en           14462267
de            2128814
fr            1465997
es            1262583
ru            1105698
it             851533
ja             818110
pt             413034
pl             376167
nl             366933
cs             363751
id             295297
sv             256776
uk             243633
wikipedia      199230
zh             196394
fa             190329
hu             183534
sk             181753
ca             165729
ro             153050
ar             147981
vi             146623
tr             134869
he             134776
fi             120354
no             116502
bg             108988
commons        104387
el             103774
ko              92725
da              87090
eo              76502
sr              72177
simple          67200
donate          61846
hr              61382
et              60739
th              60353
eu              59642
Name: url, dtype: int64

In [33]:
df2.groupby("wikidb")["domain"].nunique().sort_values(ascending=False).head(40)

wikidb
en           940239
de           155887
fr            96875
es            88903
ja            56651
it            55329
ru            53219
nl            33953
pt            27350
pl            23012
id            20732
cs            16336
sv            14716
fa            12023
vi            11190
zh            10362
hu             8256
tr             6983
no             6939
ro             6804
fi             6562
he             6323
simple         6179
da             6085
ar             6015
uk             5878
ca             5276
commons        5271
el             5163
www            5133
sk             3873
th             3645
bg             3641
wikipedia      3412
ko             2735
hr             2389
lt             2122
sr             1841
hi             1782
translate      1764
Name: domain, dtype: int64

In [34]:
df2.groupby("wikidb")["domain"].nunique().sort_values(ascending=False).sum()

1781825

In [20]:
df2 = pd.read_parquet(EMBEDDING_DIR + f"web2wiki/20210201/en/url_mentions.parquet")

In [22]:
# df2.groupby("wikidb")["domain"].nunique().sort_values(ascending=False)

wikidb
en    942273
Name: domain, dtype: int64

In [7]:
df3 = df2[df2["wiki_link"].str.contains("/wiki/")]

In [8]:
df3 = df3.drop_duplicates()

In [10]:
title_counts = df3.groupby("title")["subset"].count().sort_values(ascending=False)
domain_counts = df3.groupby("domain")["subset"].count().sort_values(ascending=False)

In [ ]:
title_counts.to_csv("/scratch/venia/web2wiki/data/processed/title_counts.csv")
domain_counts.to_csv("/scratch/venia/web2wiki/data/processed/domain_counts.csv")


In [15]:
db_counts = df3.groupby("wikidb")["subset"].count().sort_values(ascending=False)

In [19]:
domain_counts.to_csv("/scratch/venia/web2wiki/data/processed/wiki_db_counts.csv")


In [2]:
domain_counts = pd.read_csv("/scratch/venia/web2wiki/data/processed/wiki_db_counts.csv")

In [13]:
bias_scores = pd.read_csv("data/bias_scores.csv")

In [14]:
# examine the topics that are most shared
# the quality of the shares typically compared to all of wiki ? 

,domain,url,wiki_link,subset,wiki_no_subset,wikidb,title
4,agorapoliton.gr,http://agorapoliton.gr/index.php?Itemid=62&cat...,https://en.wikipedia.org/wiki/Economy_of_North...,,https://en.wikipedia.org/wiki/Economy_of_North...,en,Economy of North Korea
5,amipsyche.com,http://amipsyche.com/6-romantic-ideas-for-newl...,https://en.wikipedia.org/wiki/Newlywed,,https://en.wikipedia.org/wiki/Newlywed,en,Newlywed
6,analysebrassens.com,http://analysebrassens.com?analysebrassens=d74...,http://fr.wikipedia.org/wiki/Homosexualit%C3%A...,#Pers.C3.A9cutions_des_r.C3.A9gimes_totalitaires,http://fr.wikipedia.org/wiki/Homosexualit%C3%A9,fr,Homosexualité
7,animejima.xyz,http://animejima.xyz/2008%E5%B9%B4%EF%BC%88%E5...,https://ja.wikipedia.org/wiki/%E3%81%A8%E3%81%...,,https://ja.wikipedia.org/wiki/%E3%81%A8%E3%81%...,ja,とある魔術の禁書目録 (アニメ)
9,antifa-info.at,http://antifa-info.at/news/news2020.html,https://de.wikipedia.org/wiki/Ferdinand_von_Sa...,,https://de.wikipedia.org/wiki/Ferdinand_von_Sa...,de,Ferdinand von Sammern-Frankenegg
...,...,...,...,...,...,...,...
99353386,wirelessinfo.be,http://www.wirelessinfo.be/mikrotik-draadloos-...,https://nl.wikipedia.org/wiki/Effective_radiat...,,https://nl.wikipedia.org/wiki/Effective_radiat...,nl,Effective radiated power
99353387,wirelessinfo.be,http://www.wirelessinfo.be/mikrotik-draadloos-...,https://en.wikipedia.org/wiki/Customer-premise...,,https://en.wikipedia.org/wiki/Customer-premise...,en,Customer-premises equipment
99353388,wirelessinfo.be,http://www.wirelessinfo.be/mikrotik-draadloos-...,https://nl.wikipedia.org/wiki/Service_Set_Iden...,,https://nl.wikipedia.org/wiki/Service_Set_Iden...,nl,Service Set Identifier
99353389,wirelessinfo.be,http://www.wirelessinfo.be/mikrotik-draadloos-...,https://en.wikipedia.org/wiki/Wireless_Interne...,,https://en.wikipedia.org/wiki/Wireless_Interne...,en,Wireless Internet service provider


In [2]:
import os 
import pandas as pd 



In [15]:
df = pd.read_csv("/scratch/venia/web2wiki/data/page_views/pageviews-20210201-000000.gz", sep = " ", header = None, names = ["db", "title", "views","download"])
df2 = pd.read_csv("/scratch/venia/web2wiki/data/page_views/pageviews-20210202-000000.gz", sep = " ", header = None, names = ["db", "title", "views","download"])

In [16]:
df.drop("download", axis = 1,inplace=True)
df2.drop("download", axis = 1,inplace=True)



In [32]:
df = df[~df["db"].str.contains("[a-z]*.m")]
df2 = df2[~df2["db"].str.contains("[a-z]*.m")]

In [33]:
import time 
t = time.time()
test = df.merge(df2, on = ["db", "title"], how = "outer").fillna(0).set_index(["db", "title"]).sum(axis=1)
print(time.time() - t)

18.925930500030518


In [38]:
test = test.reset_index()
df3 =  test.merge(df2, on = ["db", "title"], how = "outer").fillna(0).set_index(["db", "title"]).sum(axis=1)

In [39]:
df3

db    title              
aa    -                       6.0
      Main_Page              20.0
      Special:ActiveUsers     1.0
      Special:ListFiles       2.0
      Special:ListUsers       1.0
                             ... 
pt.d  0                       1.0
ro    0                       2.0
ru    0                       1.0
tr    0                       2.0
zh    0                       1.0
Length: 4010054, dtype: float64

In [13]:
df.sort_values(by = "views")

,db,title,views,download
4984890,pl,Plik:Acetylcholine.svg,1,0
2467924,en.m,Klidson_Abreu,1,0
2467926,en.m,Kliff_(band),1,0
4632349,ja.m,パク・ソルミ,1,0
2467928,en.m,Klim_(powdered_milk),1,0
...,...,...,...,...
431542,de,Wikipedia:Hauptseite,23293,0
2797835,en.m,Royal_Rumble_(2021),33560,0
1647561,en,Special:Search,35758,0
1381265,en,Main_Page,87377,0


In [5]:

import sys
sys.path.append("/scratch/venia/web2wiki/")
from settings import DATA_DIR
import glob

files = glob.glob(DATA_DIR + "page_views/2021-02/pageviews-*")
   

In [19]:
schema = "wikidb STRING, title STRING, views INT, download INT"

In [20]:
df = spark.read.option("delimiter", " ").option("header", "false").csv(files, schema = schema)

In [29]:
df.take(3)

NameError: name 'df' is not defined

In [40]:
grouped = df.groupBy(["wikidb", "title"]).agg(F.sum("views").alias("views"))

In [ ]:
grouped.write.parquet("page_views_2021-02/all_views.parquet")

In [3]:
grouped2 = spark.read.load("page_views_2021-02/all_views.parquet")

[Row(wikidb='en', title='"ZHARA"_International_Music_Festival', views=61),
 Row(wikidb='en', title='(In_My)_Solitude', views=422),
 Row(wikidb='en', title="...And_the_Bag's_in_the_River", views=1185),
 Row(wikidb='en', title='.gm', views=145),
 Row(wikidb='en', title='10000-Year_Clock', views=14),
 Row(wikidb='en', title='106th_United_States_Congress', views=3223),
 Row(wikidb='en', title='10cc_(album)', views=2017),
 Row(wikidb='en', title='111th_Aviation_Regiment_(United_States)', views=144),
 Row(wikidb='en', title='11_mm_caliber', views=1821),
 Row(wikidb='en', title='124_(number)', views=348)]

In [188]:
files = next(files_partitioned)

In [214]:

print("==================")
print(f"We are on partition {i}")
t = time.time()
data = spark.read.load(files)


We are on partition 20


In [203]:
data = data.withColumn("wiki_links", extract_soup(F.col("content")))


In [207]:
selected = run(data)
# selected.write.parquet(f"/scratch/venia/web2wiki/data/wikilinks_extracted_soup/html_embedding{i}.parquet", mode = "overwrite")
print("It took {:.2f} seconds to write one chunk.".format(time.time() - t))


It took 1.58 seconds to write one chunk.


In [215]:
embedding = data.select(extract_soup("content"))

In [216]:
embedding.take(3)

22/07/06 20:48:09 ERROR Executor: Exception in task 0.0 in stage 25.0 (TID 49)4]
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/ipykernel_408949/4140495182.py", line 9, in extract_soup
  File "/home/veselovs/miniconda3/lib/python3.9/site-packages/pyspark/sql/udf.py", line 199, in wrapper
    return self(*args)
  File "/home/veselovs/spark-3.2.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/sql/udf.py", line 177, in __call__
    judf = self._judf
  File "/home/veselovs/spark-3.2.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/sql/udf.py", line 161, in _judf
    self._judf_placeholder = self._create_judf()
  File "/home/veselovs/spark-3.2.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/sql/udf.py", line 167, in _create_judf
    spark = SparkSession.builder.getOrCreate()
  File "/home/veselovs/spark-3.2.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/sql/session.py", line 234, in getOrCreate
    return session
  File "/home/veselovs/spark-3.2.1-bi

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/tmp/ipykernel_408949/4140495182.py", line 9, in extract_soup
  File "/home/veselovs/miniconda3/lib/python3.9/site-packages/pyspark/sql/udf.py", line 199, in wrapper
    return self(*args)
  File "/home/veselovs/spark-3.2.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/sql/udf.py", line 177, in __call__
    judf = self._judf
  File "/home/veselovs/spark-3.2.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/sql/udf.py", line 161, in _judf
    self._judf_placeholder = self._create_judf()
  File "/home/veselovs/spark-3.2.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/sql/udf.py", line 167, in _create_judf
    spark = SparkSession.builder.getOrCreate()
  File "/home/veselovs/spark-3.2.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/sql/session.py", line 234, in getOrCreate
    return session
  File "/home/veselovs/spark-3.2.1-bin-hadoop2.7/python/lib/p

22/07/06 20:48:09 WARN TaskSetManager: Lost task 2.0 in stage 25.0 (TID 51) (10.90.38.15 executor driver): TaskKilled (Stage cancelled)
